# JESS Web Users Functions API Example
This notebook provides a demonstration of the user's functions of the JESS Web API. For documentation of the API, please visit: [JESS Web API](http://www.jeplus.org/wiki/doku.php?id=docs:jea:jea_api "Go to JEA API docs"). 

Let's start with loading the necessary package and testing the connection to the server. The "[Requests: HTTP for Humans](http://python-requests.org "Go to Python Requests website")" is shown here, although other http packaages can be used too. 

After loading the Request package, we can run the Info command. Info is the simplest command and does not require authentication, so it works also if you copy and paste the URL to a browser, or [click here](https://api.ensims.com/users/api/info "Try Info command in browser").

In [1]:
# Use requests (see https://requests.readthedocs.io/en/master/)
import requests

# API endpoint URL
ApiBase = 'https://api.ensims.com/'
# ApiBase = 'https://localhost:8443/'
UserApi = ApiBase + 'users/api/'

# Test connection
r = requests.get(UserApi + 'info')
r.json()

{'Title': 'Users Web API',
 'Description': 'JESS User Management API provided by ENSIMS Ltd.',
 'Major': 1,
 'Minor': 0,
 'Revision': 0,
 'Release': 'beta',
 'Update': 0,
 'Notice': '(C) 2017, Energy Simulation Solutions Ltd. All rights reserved.'}

Requests will make a GET request to the given URL, and store the returned data in 'r'. Most of the data returned by the users API are in JSON format, as shown above. You can access individual field like this:

In [2]:
r.json()['Title']

'Users Web API'

## Authentication

In order to do anything useful, you must first create an account and log on to the system by acquiring a session token with your credentials. You can create a JESS Web account on [app.ensims.com](https://app.ensims.com/ "Go to app.ensims.com"). It is possible to create an account using the API functions only. Please refer to the documentation for details.

Once you have got an account, use the auth command to log on to the system. The auth command is a POST request with appropriate headers and the user credentials in JSON format as the data body. Here is how you do it. **Please remember to replace the login email with your own.** Also, since your password will be sent in clear text, please make sure that you do NOT show it in any publically accessible scripts, and use only the HTTPS protocol when accessing the JESS Web platform.

In this example, we use another library for typing in the password.

In [4]:
# Get log in credential
import getpass
user_email = "yi@jeplus.org"
password = getpass.getpass("Password for " + user_email + ": ")

# Set header and body of the POST request
headers = {'Content-Type': 'application/json'}
body = {"email": user_email, "password": password}

# Send request
r = requests.post(UserApi + 'auth', headers=headers, json=body)

# Keep the cookies
cookies = r.cookies

# Check data returned by JEA
r.json()

Password for yi@jeplus.org:  ········


{'ok': True,
 'status': 'Logged in successfully!',
 'jwt': 'Session token in cookie',
 'user': 'Yi',
 'role': 'pro',
 'email': 'yi@jeplus.org'}

The JESS Web platform uses JSON Web Tokens (JWT) as the authentication mechanism. If logged on successfully, a JWT key used as the session token will be returned by the service as a cookie. The session token needs to be kept safe and sent with any commands requiring authentication. Here is how you get the session token cookie and view its contents. Please note that the token has a limited life span of a few hours, during which time it must be kept a secrete.

To check if a session is still valid, we can use the 'checkin' command. This command also demonstrates how the session token cookie being used for authentication in the commands that require it. If 'checkin' with the existing token is successful, a new token will be returned hence we can extend the life span of the current session. Just remember to update the stored cookies.

In [7]:
# Check-in
r = requests.post(UserApi + 'checkin', cookies=cookies)

# Update the copy of the key if successful
if r.json()['ok']:
    cookies = r.cookies

# Show the response
r.json()

{'ok': True,
 'status': 'Session renewed successfully!',
 'jwt': 'Session key cookie',
 'user': 'Yi',
 'role': 'user',
 'email': 'yi@jeplus.org'}

## Check and update account details

The 'user' command is for retrieve (GET) and update (POST) user account details, as shown in the following examples.

In [13]:
# Send command
r = requests.get(UserApi + 'user', cookies=cookies)

r.json()

{'title': 'Dr',
 'firstname': 'Yi',
 'middlename': None,
 'surname': 'Zhang',
 'nickname': 'Yi',
 'position': None,
 'company': 'ENSIMS',
 'email': 'yi@jeplus.org',
 'tel': None,
 'address': None,
 'postcode': None,
 'country': None,
 'website': None,
 'twitter': None,
 'linkedin': None,
 'use_jess': False,
 'jess_user': 'TestUser',
 'jess_pwd': 'hidden'}

Now, let's set/change some details.

In [9]:
user = r.json()
user['title'] = 'Dr'
user['firstname'] = 'Yi'
user['surname'] = 'Zhang'
user['company'] = 'ENSIMS'

# Send command
r = requests.post(UserApi + 'user', json=user, cookies=cookies)

r.json()

{'ok': True,
 'status': 'Account details updated successfully!',
 'session': None}

Check the account details again to see that the changes have been applied. 

In [10]:
# retrieve user account details again to check
r = requests.get(UserApi + 'user', cookies=cookies)

# Show the returned status
r.json()

{'title': 'Dr',
 'firstname': 'Yi',
 'middlename': None,
 'surname': 'Zhang',
 'nickname': 'Yi',
 'position': None,
 'company': 'ENSIMS',
 'email': 'yi@jeplus.org',
 'tel': None,
 'address': None,
 'postcode': None,
 'country': None,
 'website': None,
 'twitter': None,
 'linkedin': None,
 'use_jess': False,
 'jess_user': 'test',
 'jess_pwd': 'hidden'}

And, that's about it. There are three special fields, 'use_jess', 'jess_user' and 'jess_pwd', to link the online simulation service provided by ENSIMS to the user's account. This will be detailed in the JESS API example.

## Upload, download, and view files

We can upload files to a storage area linked to this user account. These files can be used for simulation (see JESS API) or optimisation projects (see JEA API) later. With certain file types, e.g. the EPW, ESO and IDF files, we can also use this facility as a viewer. 

First, the GET files/ comamnd retrieves the list of contents of a folder in the storage area in a tree structure:

In [28]:
# retrieve files in a particular folder
folder = "project3/weather"
r = requests.get(UserApi + 'files/' + folder, cookies=cookies)

# Show the returned status
r.json()

c:\bin\python38\lib\site-packages\urllib3\connection.py:388: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(


{'name': '/project3/weather',
 'fileName': '/project3/weather',
 'open': True,
 'nocheck': True,
 'children': [{'name': 'GBR_London.Gatwick.037760_IWEC.epw (1.55 MB)',
   'fileName': 'GBR_London.Gatwick.037760_IWEC.epw',
   'open': False,
   'nocheck': False,
   'children': [],
   'fileSize': 1547758}],
 'fileSize': 0}

The same command downloads the file if the paths are pointing to a file instead of a folder:

In [29]:
# retrieve a file in a particular folder
folder = "project3/parameters.csv"
r = requests.get(UserApi + 'files/' + folder, cookies=cookies)

# Show the contents of the downloaded file
r.text

c:\bin\python38\lib\site-packages\urllib3\connection.py:388: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(


'# Parameter list for project: 0 (exported at 01/09/15 14:46)\r\n# Note: this list contains only the first branch of the parameter tree.\r\n# Parameter definitions in a csv file. Column headings are as below\r\n# ID, Name, Parameter Type, Description, Search String, Value Type, Value String, Selected Value Index\r\n#           {0}                                         {0, 1, 2}                 {0, .... depending on number of values}\r\n# \r\nP1,Orientation,0,"Orientation of the building",@@orientation@@,0,[0 :45 :359 ] & {101} ^ {45, 135},0\r\nP2,Outside Air Rate,0,"Outside Air Rate [m3/s/person]",@@Outside_Air@@,1,@sample(n, 0.008, 0.003, 5),0\r\nP3,Occupancy density,0,"occupancy density [people / m^2]",@@Occ@@,1,@file(P3_sample_EXP.txt),0\r\nP4,Equipment power,0,"Equipment power as a function of occupancy density",@@Eq@@,1,@calc(-400*P3*P3+220*P3+2),0\r\n'

To upload a file to a specific folder, we can use POST files/

In [32]:
# upload a file to a particular folder
folder = "test/0"
files = {'file': open('c:\windows\win.ini', 'rb')}

# POST with files
r = requests.post(UserApi + 'files/' + folder, files=files, cookies=cookies)

# Show the returned status
r.json()

c:\bin\python38\lib\site-packages\urllib3\connection.py:388: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(


{'ok': True,
 'status': 'File(s) uploaded successfully',
 'fileHandle': 'test\\0',
 'callback': '/users/api/files/test\\0'}

When a zip archive containing multiple files and folders is uploaded, its contents will be extracted automatically and stored in the user's upload area, as shown in the example here. Please note the returned file handle may be the target folder in the user's area, or the root folder of the zip file residing in the target folder, depending on how the zip archive is created.

In [36]:
# upload a file to a particular folder
folder = ""
files = {'file': open('C:\zyyz\JESS_Client_v3.0.2\example_jep_v81.zip', 'rb')}

# POST with files
r = requests.post(UserApi + 'files/' + folder, files=files, cookies=cookies)

# Show the returned status
r.json()

c:\bin\python38\lib\site-packages\urllib3\connection.py:388: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(


{'ok': True,
 'status': 'File(s) uploaded successfully',
 'fileHandle': 'example_jep_v81',
 'callback': '/users/api/files/example_jep_v81'}

To delet a file or a folder with everything in it, use the DELETE files/ command:

In [35]:
# delete a file from the given folder
# filepath = "test/0/win.ini"
filepath = "example_jep_v81/"

# DELETE with files
r = requests.delete(UserApi + 'files/' + filepath, cookies=cookies)

# Show the returned status
r.json()

c:\bin\python38\lib\site-packages\urllib3\connection.py:388: SubjectAltNameWarning: Certificate for localhost has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/urllib3/urllib3/issues/497 for details.)
  warnings.warn(


{'ok': True, 'status': 'example_jep_v81/ has been deleted'}

And, finally, JESS Web currently provides viewers for EPW, ESO and IDF files. Examples can be found in the JESS_API examples notebook. 

## Summary

This tutorial document shows the main functions of the Users API of the JESS Web platform. You can find more details in the [docs](https://www.jeplus.org/wiki/doku.php?id=docs:jea:jea_api#summary "Go to JEA docs"). 
